In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import fiona

In [2]:
cnty = {'Alameda': 1.0,
'Contra Costa': 13.0,
'Marin': 41.0,
'Napa': 55.0,
'San Francisco': 75.0,
'San Mateo': 81.0,
'Santa Clara': 85.0,
'Solano': 95.0,
'Sonoma': 97.0}

ctyMap = pd.DataFrame(cnty.items(), columns=['ctyName', 'ctyCode'])

## 0 Data sources

In [3]:
print(fiona.listlayers(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\Projects\PLU_analysis\PLU_analysis.gdb'))

# Input files
## Pacel 10
p10_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS Pro 2.5\Projects\PLU_analysis\PLU_analysis.gdb', layer='p10_table')

## parcel10 to pba40 basezoning code
pz10 = pd.read_csv(r'C:\Users\ywang\Box\Mine\1_UrbanSim\2020_03_06_zoning_parcels.csv')

## pba40 basezoning plu
plu10 = pd.read_csv('C:/Users/ywang/Documents/GitHub/bayarea_urbansim/data/zoning_lookup.csv')

## pba50 basezoning PLU
print(fiona.listlayers(r'C:\Users\ywang\Documents\ArcGIS\Projects\UrbanSim Data Review Option B v1c\UrbanSim Data Review Option B v1c.gdb'))
p10_plu50_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\UrbanSim Data Review Option B v1c\UrbanSim Data Review Option B v1c.gdb', layer='p10_boc_opt_B_v3_geo_yq')
#plu50 = pd.read_csv(r'C:\Users\ywang\Box\Mine\3_BASIS\urbansim_boc_2010_mod.csv')

## planned zoning scenarios
zmods = pd.read_csv('C:/Users/ywang/Box/Mine/1_UrbanSim/03_06_2020_parcels_geography.csv')

## Building data to decide parcel status
blg10 = pd.read_csv('blg10.csv')

['p10_table', 'p10_boc_v3_geo_tbl_20200311']
['p10_boc_opt_B_v3_geo_yq']


## 1 Merge data sets

### 1.1 P10 parcel zoining designations

In [4]:
# parcel geometry
p10 = p10_raw[['PARCEL_ID','APN','geom_id_s','COUNTY_ID','jurisdiction',
                           'ACRES','LAND_VALUE','pda_id','zoningmodcat']]
print(p10.shape)
display(p10.head())

# pacel to zoning code mapping
print(pz10.shape)
display(pz10.head())

p10_z10 = p10.merge(pz10, on = 'PARCEL_ID', how = 'left')
print(p10_z10.shape)
display(p10_z10.head())

# Check Number of parcels missing zoning designation
z10_missing = p10_z10.loc[p10_z10['nodev_pba40'].isnull()]
print(z10_missing.shape[0])
print(z10_missing.shape[0]/pz10.shape[0])

(1956208, 9)


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA


(1950733, 11)


,Unnamed: 0,geom_id,zoning_id,zoning,juris,prop,tablename,nodev_pba40,PARCEL_ID,nodev,juris_id
0,0,1846247885201,12202.0,115 - Residential 0-4 du/ac,102,100,alamedacountygp2006db,0,191124.0,0.0,uala
1,1,11768793521677,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0,197219.0,0.0,uala
2,2,807545210880,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0,197218.0,0.0,uala
3,3,8785012057974,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0,188301.0,0.0,uala
4,4,14057552282712,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0,188939.0,0.0,uala


(1956208, 19)


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,Unnamed: 0,geom_id,zoning_id,zoning,juris,prop,tablename,nodev_pba40,nodev,juris_id
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1771381.0,1.030511e+13,60126.0,107 - Urban Low Residential UL2,-9999.0,100.0,plu06,0.0,0.0,livr
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1362648.0,1.110735e+13,11903.0,GP-ULM,99.0,100.0,livermoregeneralplan,0.0,0.0,livr
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,307258.0,1.103018e+13,11803.0,LDR,98.0,100.0,hayward_gp_landuse,0.0,0.0,hayw
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,1737625.0,6.381678e+12,12975.0,LEA240,109.0,100.0,sonomacountygeneralplan,0.0,0.0,uson
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,273989.0,3.148755e+11,2511.0,RESM4,5.0,100.0,fremontgeneralplan,0.0,1.0,frem


5476
0.002807149927745109


### 1.2 parcel 10 with PBA40 zoning code PLU

In [5]:
# check duplicates in zoning id
plu10['id'] = plu10['id'].apply(lambda x: float(x))
plu10['jz_o'] = plu10['city'].str.cat(plu10['name'],sep=" ")
print(plu10.shape[0], len(plu10.id.unique()), len(plu10.jz_o.unique()))

# relabel p10 land plu info (used in PBA40)
cols = [i+'_10' for i in list(plu10)]
plu10.columns = cols
display(plu10.head())

# merge PBA40 plu to p10
p10_plu10 = p10_z10.merge(plu10, left_on = 'zoning_id', right_on = 'id_10', how = 'left')
display(p10_plu10.head())

# Check number of p10 records failed to find a matching PLU
display(p10_plu10.loc[p10_plu10['jz_o_10'].isnull()])
print(p10_plu10.loc[p10_plu10['jz_o_10'].isnull()].shape[0] / p10_z10.shape[0])

5156 5156 4536


,id_10,juris_10,city_10,name_10,max_far_10,max_height_10,max_dua_10,max_du_per_parcel_10,HS_10,HT_10,...,IL_10,IW_10,IH_10,RS_10,RB_10,MR_10,MT_10,ME_10,plandate_10,jz_o_10
0,2101.0,1.0,Albany,RHD,0.50,35.0,9.0,NaN,1,0,...,0,0,0,0,0,0,0,0,NaN,Albany RHD
1,2102.0,1.0,Albany,R-1,0.55,28.0,12.0,NaN,1,0,...,0,0,0,0,0,0,0,0,NaN,Albany R-1
2,2103.0,1.0,Albany,R-2,0.55,35.0,35.0,NaN,1,1,...,0,0,0,0,0,0,0,0,NaN,Albany R-2
3,2104.0,1.0,Albany,R-3,1.50,35.0,63.0,NaN,1,1,...,0,0,0,0,0,0,0,0,NaN,Albany R-3
4,2105.0,1.0,Albany,R.4,NaN,NaN,87.0,NaN,0,0,...,0,0,0,0,0,0,0,0,NaN,Albany R.4


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,Unnamed: 0,...,IL_10,IW_10,IH_10,RS_10,RB_10,MR_10,MT_10,ME_10,plandate_10,jz_o_10
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1771381.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Livermore 107 - Urban Low Residential UL2
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1362648.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Livermore GP-ULM
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,307258.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Hayward LDR
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,1737625.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Unincorporated Sonoma LEA240
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,273989.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Fremont RESM4


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,Unnamed: 0,...,IL_10,IW_10,IH_10,RS_10,RB_10,MR_10,MT_10,ME_10,plandate_10,jz_o_10
1185,580792.0,271333,14613597454578,13.0,16000,0.013816,0.000000e+00,NA,16000NANANANA,1322496.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1637,255073.0,415 000100600,15298997157075,1.0,00001,0.007685,0.000000e+00,NA,00001NANANANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1750,1323823.0,15846302,261407886174,85.0,49670,2.773525,3.467860e+07,MVW2,49670MVW2cr2NANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1833,2054503.0,,2054503,0.0,00085,0.000000,0.000000e+00,NA,00085NANANANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1956,1220739.0,090160500,12373840766913,81.0,17918,0.059405,0.000000e+00,NA,17918NANANANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955104,651123.0,334654,13367406559562,13.0,57456,0.115417,2.306800e+04,PIT1,57456PIT1bart3NANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955850,680555.0,44228,9247454957509,13.0,49187,0.609596,0.000000e+00,NA,49187NANANANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955895,1411792.0,28213001,3484331839412,85.0,68000,7.325086,3.932012e+06,SJO14,68000SJO14lrt3NANA,582835.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1956145,1414620.0,28422021,8329617969303,85.0,68000,2.655408,0.000000e+00,NA,68000NAlrt3NANA,1305453.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0.0034745793903306807


### 1.3 P10 with BASIS BOC

In [7]:
plu50 = p10_plu50_raw.loc[:,['PARCEL_ID','me','mt', 'mr', 'rb', 'rs', 'ih', 'iw', 'il', 'sc', 'ho', 'of', 'hm', 'ht', 'hs',
                       'max_height','max_dua','max_far','plu_id','plu_jurisdiction','plu_description']]

print(plu50.shape)

# relabel BASIS land plu info (to use in PBA50)
cols2 = [i+'_18' for i in list(plu50)]
plu50.columns = cols2
display(plu50.head())

# merge PBA50 plu to p10
p10_plus = p10_plu10.merge(plu50, left_on = 'PARCEL_ID', right_on = 'PARCEL_ID_18', how = 'left')
print(p10_plus.shape)

p10_plus.drop(columns = ['zoning','tablename','Unnamed: 0','id_10','name_10','plandate_10','jz_o_10','PARCEL_ID_18'],inplace = True)
display(p10_plus.head())

(1956208, 21)


,PARCEL_ID_18,me_18,mt_18,mr_18,rb_18,rs_18,ih_18,iw_18,il_18,sc_18,...,of_18,hm_18,ht_18,hs_18,max_height_18,max_dua_18,max_far_18,plu_id_18,plu_jurisdiction_18,plu_description_18
0,229116.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,0.00,0.00,fc5f982a-40e3-452f-a660-f5f80b7d0b24,Livermore,Planned Development
1,244166.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,35.0,14.00,0.35,4d5ddd2e-464a-4861-9c1f-88941a0b5676,Livermore,Residential
2,202378.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,NaN,0.00,0.00,5a6d3363-b377-4431-8ac8-35aa1593225d,Hayward,Planned Development
3,2004420.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,35.0,0.05,0.00,3e798de2-3854-40a6-b29d-bbc9eadbeb01,Unincorporated Sonoma,Resources and Rural Development
4,340332.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30.0,1.00,0.01,9b689549-939f-4288-aad7-d3584fef122f,Fremont,Planned District


(1956208, 64)


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,geom_id,...,of_18,hm_18,ht_18,hs_18,max_height_18,max_dua_18,max_far_18,plu_id_18,plu_jurisdiction_18,plu_description_18
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1.030511e+13,...,0.0,0.0,0.0,0.0,NaN,0.00,0.00,fc5f982a-40e3-452f-a660-f5f80b7d0b24,Livermore,Planned Development
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1.110735e+13,...,0.0,0.0,0.0,0.0,35.0,14.00,0.35,4d5ddd2e-464a-4861-9c1f-88941a0b5676,Livermore,Residential
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,1.103018e+13,...,1.0,0.0,0.0,0.0,NaN,0.00,0.00,5a6d3363-b377-4431-8ac8-35aa1593225d,Hayward,Planned Development
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,6.381678e+12,...,0.0,0.0,0.0,1.0,35.0,0.05,0.00,3e798de2-3854-40a6-b29d-bbc9eadbeb01,Unincorporated Sonoma,Resources and Rural Development
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,3.148755e+11,...,NaN,NaN,NaN,NaN,30.0,1.00,0.01,9b689549-939f-4288-aad7-d3584fef122f,Fremont,Planned District


### 1.4 Bring in Building data (b10) to determine parcel characteristics

In [8]:
print(blg10.shape[0], len(blg10.building_id.unique()), len(blg10.parcel_id.unique()))
display(blg10.head())

# Assign parcel characteristics

# merge builing and parcel data w/ Outer-join
b10_p10 = blg10.merge(p10[['PARCEL_ID']],left_on = 'parcel_id',right_on = 'PARCEL_ID', how = 'outer')
print(b10_p10.shape)

# sum all values for multiple buildings within one parcel
pb10_v = b10_p10.groupby(['PARCEL_ID'])['improvement_value','residential_units','residential_sqft','non_residential_sqft',
                                      'building_sqft','redfin_sale_price','costar_rent'].sum().reset_index()

# chose the earliest built year for multiple buildings within one parcel
pb10_yr = b10_p10.groupby(['PARCEL_ID'])['year_built','building_id'].min().reset_index()

# parcel vacancy based on building type
b10_p10['dType'] = b10_p10['development_type_id']
blg10.loc[blg10['development_type_id'] == 0, 'dType'] = 'Vacant'
blg10.loc[blg10['development_type_id'] == 15, 'dType'] = 'Vacant'
pb10_vacent = b10_p10.loc[b10_p10['dType'] == 'Vacant'][['PARCEL_ID','dType']]

# merge
pb10_temp = pb10_v.merge(pb10_yr, on = 'PARCEL_ID', how = 'left').merge(pb10_vacent, on = 'PARCEL_ID', how = 'left')
print(pb10_temp.shape)
pb10_plus = p10_plus.merge(pb10_temp, on = 'PARCEL_ID', how = 'left')

# Investment-land ratio
pb10_plus['ILR'] = pb10_plus['improvement_value'] / pb10_plus['LAND_VALUE']
pb10_plus.loc[pb10_plus['LAND_VALUE'] == 0, 'ILR'] = 'n/a'

# Vacant parcels
pb10_plus['vacant'] = np.where((pb10_plus['building_id'].isnull()) | (pb10_plus['dType'] == 'Vacant') | 
                          ((pb10_plus['improvement_value'] == 0) & (pb10_plus['residential_units'] == 0) & 
                             (pb10_plus['residential_sqft'] == 0) & (pb10_plus['non_residential_sqft'] == 0) &
                             (pb10_plus['building_sqft'] == 0)), 'vacant', 'nonVacant')

pb10_plus.head()

1843351 1843351 1843292


,Unnamed: 0,building_id,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,...,res_price_per_sqft,stories,year_built,redfin_sale_price,redfin_sale_year,redfin_home_type,costar_property_type,costar_rent,id,geometry
0,0,1,742974,1,0.00,1,2029,2029.42425,0,2029.42425,...,302.769751,1,1945,NaN,NaN,NaN,NaN,NaN,1,NaN
1,1,2,744961,1,0.00,1,2029,2029.42425,0,2029.42425,...,254.429279,1,1965,NaN,NaN,NaN,NaN,NaN,2,NaN
2,2,3,1442641,1,53262.87,1,1568,1568.00000,0,1568.00000,...,183.474166,1,1964,NaN,NaN,NaN,NaN,NaN,3,NaN
3,3,4,190969,2,245000.00,0,0,1266.00000,1595,1266.00000,...,0.000000,2,1992,340000.0,2003.0,Condo/Coop,NaN,NaN,4,NaN
4,4,5,308709,2,283500.00,0,0,1513.00000,1513,1513.00000,...,0.000000,1,1978,442000.0,2004.0,Condo/Coop,NaN,NaN,5,NaN


(1956269, 22)


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  from ipykernel import kernelapp as app
C:\Users\ywang\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


(1956208, 10)


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,geom_id,...,residential_units,residential_sqft,non_residential_sqft,building_sqft,redfin_sale_price,year_built,building_id,dType,ILR,vacant
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1.030511e+13,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,n/a,vacant
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1.110735e+13,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,n/a,vacant
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,1.103018e+13,...,20.0,101000.0,0.0,101000.0,1007250.0,2009.0,15681.0,NaN,0,nonVacant
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,6.381678e+12,...,0.0,0.0,0.0,0.0,0.0,1965.0,17798.0,NaN,0.812491,nonVacant
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,3.148755e+11,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,n/a,vacant


### 1.5 Bring in zoning scenarios data

In [9]:
print(zmods.shape)
zmods.columns = list(zmods)[:-2] + ['nodev_pba50','jurisdiction_id']
display(zmods.head())

# merge parcel data with zoning mods
pb10_plus_zmods = pb10_plus.merge(zmods, on = 'geom_id', how = 'left')
print(pb10_plus_zmods.shape)
display(pb10_plus_zmods.head())

(1956208, 26)


,geom_id,jurisdiction_id_old,pda_id,tpp_id,exp_id,opp_id,zoningmodcat,perffoot,perfarea,urbanized,...,juris_id,gg_id,tra_id,sesit_id,ppa_id,exp2020_id,exsfd_id,pba50zoningmodcat,nodev_pba50,jurisdiction_id
0,10305106092872,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,1,...,livr,NaN,NaN,HRADR,NaN,in,NaN,livrNANAHRADRNAinNA,0,41992
1,11107351665227,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,1,...,livr,NaN,NaN,DR,NaN,in,NaN,livrNANADRNAinNA,0,41992
2,11030175960628,33000,NaN,NaN,NaN,NaN,33000NANANANA,1,0,0,...,hayw,NaN,NaN,NaN,NaN,in,NaN,haywNANANANAinNA,0,33000
3,6381677629073,97,NaN,NaN,NaN,NaN,00097NANANANA,0,0,0,...,uson,NaN,NaN,DR,NaN,out,NaN,usonNANADRNAoutNA,0,97
4,314875459798,26000,NaN,b1,NaN,NaN,26000NAb1NANA,1,1,1,...,frem,NaN,NaN,HRADR,NaN,in,NaN,fremNANAHRADRNAinNA,1,26000


(1956208, 92)


,PARCEL_ID_x,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id_x,zoningmodcat_x,geom_id,...,juris_id_y,gg_id,tra_id,sesit_id,ppa_id,exp2020_id,exsfd_id,pba50zoningmodcat,nodev_pba50,jurisdiction_id
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1.030511e+13,...,livr,NaN,NaN,HRADR,NaN,in,NaN,livrNANAHRADRNAinNA,0.0,41992.0
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1.110735e+13,...,livr,NaN,NaN,DR,NaN,in,NaN,livrNANADRNAinNA,0.0,41992.0
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,1.103018e+13,...,hayw,NaN,NaN,NaN,NaN,in,NaN,haywNANANANAinNA,0.0,33000.0
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,6.381678e+12,...,uson,NaN,NaN,DR,NaN,out,NaN,usonNANADRNAoutNA,0.0,97.0
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,3.148755e+11,...,frem,NaN,NaN,HRADR,NaN,in,NaN,fremNANAHRADRNAinNA,1.0,26000.0


### 1.6 Export BOC data for mapping

In [10]:
p10_plu_boc = pb10_plus_zmods.loc[:,['PARCEL_ID_x','COUNTY_ID','juris_id_y','plu_id_18','plu_jurisdiction_18','plu_description_18',
                    'max_far_10','max_dua_10','max_dua_18','max_far_18',
                    'HS_10','HT_10','HM_10','OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10',
                    'me_18','mt_18','mr_18','rb_18','rs_18','ih_18','iw_18','il_18','sc_18','ho_18','of_18','hm_18','ht_18','hs_18']]

In [11]:
p10_plu_boc.to_csv('p10_plu_boc.csv',index = False)

In [12]:
p10_plu = pb10_plus_zmods.loc[:,['PARCEL_ID_x','juris_id_y','max_far_10','max_dua_10',
                    'HS_10','HT_10','HM_10','OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10']]

In [13]:
p10_plu.to_csv('p10_plu.csv',index = False)

## 2 Capacity statistics

### 2.1 Allowed Development Type Statistics

In [14]:
cty = ctyMap
cty.set_index('ctyCode',inplace = True)
#cty.columns = ['ctyName','ctyCode']

In [15]:
p10_plu_boc_clean = p10_plu_boc.loc[p10_plu_boc['COUNTY_ID'] > 0]

In [16]:
dfs = []
for i in [['MR_10','mr_18'],['RS_10','rs_18'],['HS_10','hs_18'],['HT_10','ht_18'],['HM_10','hm_18'],['OF_10','of_18'],
          ['HO_10','ho_18'],['SC_10','sc_18'],['IL_10','il_18'],['IW_10','iw_18'],['IH_10','ih_18'],['RB_10','rb_18'],
          ['MT_10','mt_18'],['ME_10','me_18']]:
    df = p10_plu_boc_clean[['COUNTY_ID']+ i ].groupby(['COUNTY_ID']).sum().reset_index()
    df.set_index('COUNTY_ID',inplace = True)
    dfs.append(df)

In [17]:
plu_boc_comp = pd.concat([cty] + dfs, axis=1,join='inner')
plu_boc_comp

,ctyName,MR_10,mr_18,RS_10,rs_18,HS_10,hs_18,HT_10,ht_18,HM_10,...,IW_10,iw_18,IH_10,ih_18,RB_10,rb_18,MT_10,mt_18,ME_10,me_18
1.0,Alameda,18346.0,56316.0,59353.0,96821.0,308544.0,260171.0,163145.0,82943.0,117971.0,...,8028.0,39620.0,4095.0,27179.0,6899.0,20762.0,21166.0,72510.0,11707.0,74732.0
13.0,Contra Costa,4453.0,49661.0,18504.0,97835.0,273712.0,208426.0,115984.0,102389.0,36686.0,...,3911.0,60297.0,1482.0,5303.0,3962.0,77607.0,6882.0,85904.0,5733.0,77154.0
41.0,Marin,1890.0,14476.0,3134.0,23632.0,80289.0,64887.0,25842.0,20463.0,14291.0,...,1232.0,881.0,618.0,56.0,762.0,9744.0,2350.0,16594.0,2308.0,16288.0
55.0,Napa,478.0,1265.0,1565.0,7155.0,39684.0,39042.0,12947.0,26872.0,10671.0,...,8508.0,2940.0,66.0,165.0,1209.0,4823.0,1058.0,1803.0,1043.0,1989.0
75.0,San Francisco,13448.0,14761.0,10785.0,14676.0,11887.0,131699.0,143283.0,55148.0,71920.0,...,2621.0,2282.0,77.0,416.0,1753.0,411.0,14090.0,12204.0,3442.0,706.0
81.0,San Mateo,7433.0,23907.0,9275.0,50336.0,170416.0,153051.0,83479.0,49884.0,55032.0,...,5663.0,8701.0,3571.0,1250.0,5907.0,14476.0,4827.0,26964.0,4002.0,17958.0
85.0,Santa Clara,21875.0,33989.0,193675.0,72773.0,364912.0,358767.0,273546.0,257073.0,50022.0,...,7397.0,32423.0,2901.0,4321.0,2852.0,20288.0,28739.0,38043.0,8435.0,43741.0
95.0,Solano,2092.0,11654.0,7321.0,46673.0,109227.0,74687.0,57421.0,42282.0,16018.0,...,1471.0,5463.0,1613.0,5174.0,4310.0,7122.0,2918.0,8317.0,2834.0,9592.0
97.0,Sonoma,5551.0,69573.0,53089.0,88673.0,154446.0,146069.0,81427.0,89170.0,65463.0,...,3471.0,20958.0,895.0,15391.0,3501.0,25366.0,6088.0,52073.0,5685.0,66665.0


### 2.2 Caculate Build out capacity for each parcel

In [18]:
# select needed fields
plu_main = pb10_plus_zmods.loc[:,['COUNTY_ID','juris_id_y','zoning_id','geom_id_s','ACRES',
                    'max_far_10','max_dua_10','max_dua_18','max_far_18',
                    'HS_10','HT_10','HM_10','OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10',
                    'me_18','mt_18','mr_18','rb_18','rs_18','ih_18','iw_18','il_18','sc_18','ho_18','of_18','hm_18','ht_18','hs_18',
                    'year_built','ILR','vacant','pba50zoningmodcat','nodev_pba40','nodev_pba50']]

# Convert all types to numeric to enable calculation
l = ['HS_10','HT_10','HM_10','OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10',
     'me_18','mt_18','mr_18','rb_18','rs_18','ih_18','iw_18','il_18','sc_18','ho_18','of_18','hm_18','ht_18','hs_18',
     'max_far_10','max_dua_10','max_dua_18','max_far_18']

for i in l:
    plu_main[i] = pd.to_numeric(plu_main[i], errors='coerce')

# fill 'NaN' with 0
plu_main.update(plu_main[l].fillna(0))

In [19]:
# Calculate BOC Based on PBA40 PLU

## A parcel is 'allowNonRes' is at least one of the non-residential development types is allowed; then FAR calculations apply
plu_main['allowNonRes_10'] = plu_main[['OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10']].sum(axis=1) > 0

## A parcel is 'allowRes' is at least one of the residential development types is allowed; then DUA calculations apply
plu_main['allowRes_10'] = plu_main[['HS_10','HT_10','HM_10']].sum(axis=1) > 0

plu_main['units_10'] = plu_main['ACRES'] * plu_main['max_dua_10']
plu_main.loc[(plu_main['allowRes_10'] is False) | (plu_main['nodev_pba40'] == 1) ,'units_10'] = 0
plu_main['sf_10'] = plu_main['ACRES'] * plu_main['max_far_10'] * 43560
plu_main.loc[(plu_main['allowNonRes_10'] is False) | (plu_main['nodev_pba50'] == 1),'sf_10'] = 0


# Calculate BOC Based on BASIS PLU

plu_main['allowNonRes_18'] = plu_main[['of_18','ho_18','sc_18','il_18','iw_18','ih_18','rs_18','rb_18','mr_18','mt_18','me_18']].sum(axis=1) > 0
plu_main['allowRes_18'] = plu_main[['hs_18','ht_18','hm_18']].sum(axis=1) > 0

plu_main['units_18'] = plu_main['ACRES'] * plu_main['max_dua_18']
plu_main.loc[(plu_main['allowRes_18'] is False) | (plu_main['hs_18'] == 1) ,'units_18'] = 0
plu_main['sf_18'] = plu_main['ACRES'] * plu_main['max_far_18'] * 43560
plu_main.loc[(plu_main['allowNonRes_18'] is False) | (plu_main['hs_18'] == 1),'sf_18'] = 0

display(plu_main)

,COUNTY_ID,juris_id_y,zoning_id,geom_id_s,ACRES,max_far_10,max_dua_10,max_dua_18,max_far_18,HS_10,...,nodev_pba40,nodev_pba50,allowNonRes_10,allowRes_10,units_10,sf_10,allowNonRes_18,allowRes_18,units_18,sf_18
0,1.0,livr,60126.0,10305106092872,3.360520,0.0,2.00000,0.000,0.00,1.0,...,0.0,0.0,False,True,6.721041,0.000000,False,False,0.000000,0.000000
1,1.0,livr,11903.0,11107351665227,1.294423,0.0,3.00000,14.000,0.35,0.0,...,0.0,0.0,False,True,3.883268,0.000000,False,False,18.121919,19734.769419
2,1.0,hayw,11803.0,11030175960628,14.993605,0.0,8.70000,0.000,0.00,1.0,...,0.0,0.0,False,True,130.444362,0.000000,True,False,0.000000,0.000000
3,97.0,uson,12975.0,6381677629073,316.247146,0.0,0.00417,0.050,0.00,1.0,...,0.0,0.0,False,True,1.318751,0.000000,False,True,0.000000,0.000000
4,1.0,frem,2511.0,314875459798,0.621275,0.0,23.00000,1.000,0.01,1.0,...,0.0,1.0,True,True,14.289334,0.000000,False,False,0.621275,270.627567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956203,13.0,anti,10204.0,17158666132196,0.071424,0.0,10.00000,0.000,0.00,1.0,...,0.0,0.0,True,True,0.714239,0.000000,True,True,0.000000,0.000000
1956204,13.0,conc,10702.0,16389503450045,0.137534,0.0,10.00000,6.000,0.00,1.0,...,0.0,0.0,False,True,1.375336,0.000000,False,True,0.000000,0.000000
1956205,41.0,nova,8213.0,1496694834659,0.019658,0.4,0.00000,4.356,0.40,0.0,...,0.0,0.0,True,False,0.000000,342.523479,True,False,0.085631,342.523479
1956206,13.0,ucnc,12302.0,10694584892329,0.254764,0.0,2.90000,0.000,0.00,1.0,...,0.0,0.0,False,True,0.738815,0.000000,False,False,0.000000,0.000000


### 2.3 Build out capacity at jurisdiction and county levels

In [22]:
# BOC by jurisdiction function
def boc_j(df):
    boc_j = df.groupby(['juris_id_y'])['ACRES','units_10','units_18','sf_10','sf_18'].sum()

    boc_j['unit_diff'] = boc_j['units_18'] - boc_j['units_10']
    boc_j['sqft_diff'] = boc_j['sf_18'] - boc_j['sf_10']
    boc_j['unit_diff_pct'] = boc_j['unit_diff'] / boc_j['units_10']
    boc_j['sqft_diff_pct'] = boc_j['sqft_diff'] / boc_j['sf_10']

    for i in ['units_10','units_18','unit_diff','sf_10','sf_18','sqft_diff']:
        boc_j[i] = boc_j[i].apply(lambda x: f'{int(x):,}')
    display(boc_j)
    return boc_j

# BOC by county function
def boc_c(df):
    boc_cty = df.groupby(['COUNTY_ID'])['ACRES','units_10','units_18','sf_10','sf_18'].sum()
    boc_cty['unit_diff'] = boc_cty['units_18'] - boc_cty['units_10']
    boc_cty['sqft_diff'] = boc_cty['sf_18'] - boc_cty['sf_10']
    boc_cty['unit_diff_pct'] = boc_cty['unit_diff'] / boc_cty['units_10']
    boc_cty['sqft_diff_pct'] = boc_cty['sqft_diff'] / boc_cty['sf_10']

    for i in ['units_10','units_18','unit_diff','sf_10','sf_18','sqft_diff']:
        boc_cty[i] = boc_cty[i].apply(lambda x: f'{int(x):,}')

    boc_cty = boc_cty.reset_index()
    boc_cty = boc_cty.loc[boc_cty['COUNTY_ID'] > 0]

    boc_c = boc_cty.merge(ctyMap, left_on = 'COUNTY_ID', right_on = 'ctyCode', how = 'left')
    #boc_c.drop(columns = ['ctyCode'],inplace=True)
    display(boc_c)
    return boc_c

In [23]:
# all parcels statistics
all_boc_j = boc_j(plu_main)
all_boc_j.to_csv('all_boc_jurisdiction.csv')

all_boc_c = boc_c(plu_main)
all_boc_c.to_csv('all_boc_county.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct
juris_id_y,,,,,,,,,
alam,6576.815744,"34,053","5,975","51,852,302","53,643,769","-28,077","1,791,467",-0.824517,0.034549
alba,2685.790749,"9,803",0,"17,401,747","14,707","-9,803","-17,387,039",-1.000000,-0.999155
amer,3459.100475,"6,842",339,"40,526,893","21,329,877","-6,502","-19,197,015",-0.950360,-0.473686
anti,15626.623830,"79,272","1,992","143,859,903","55,949,944","-77,280","-87,909,958",-0.974866,-0.611080
athe,2884.446231,"2,518",0,"20,406,205",0,"-2,518","-20,406,205",-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...
vall,18923.391859,"59,916",0,"100,599,543",0,"-59,916","-100,599,543",-1.000000,-1.000000
walc,11199.444875,"50,396","12,542","52,654,022","45,955,077","-37,853","-6,698,944",-0.751118,-0.127226
wind,4000.762589,"14,103","13,795","10,472,140","17,247,510",-308,"6,775,370",-0.021862,0.646990


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,COUNTY_ID,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct,ctyName
0,1.0,494335.528614,"1,132,680","762,316","2,284,007,108","8,182,882,406","-370,364","5,898,875,297",-0.326981,2.582687,Alameda
1,13.0,446722.616106,"768,422","227,453","999,499,932","732,004,376","-540,969","-267,495,556",-0.704000,-0.267629,Contra Costa
2,41.0,371976.214899,"223,537","381,208","160,991,354","571,137,173","157,670","410,145,819",0.705345,2.547626,Marin
3,55.0,495755.032111,"464,635","70,383","703,537,212","2,142,923,481","-394,252","1,439,386,268",-0.848520,2.045928,Napa
4,75.0,22683.294626,"411,420","91,866","1,385,258,898","464,519,003","-319,553","-920,739,894",-0.776709,-0.664670,San Francisco
5,81.0,336923.887211,"394,233","149,316","955,891,855","445,848,673","-244,916","-510,043,181",-0.621248,-0.533578,San Mateo
6,85.0,797516.101744,"1,028,659","626,446","7,913,791,752","803,563,429,514","-402,213","795,649,637,762",-0.391007,100.539623,Santa Clara
7,95.0,529122.858613,"239,209","89,681","660,264,947","278,662,393","-149,528","-381,602,553",-0.625095,-0.577954,Solano
8,97.0,993067.346741,"299,285","177,907","1,590,310,809","354,467,940","-121,378","-1,235,842,868",-0.405561,-0.777108,Sonoma


In [24]:
# vacant parcel statistics

p_vac = plu_main.loc[plu_main.vacant == 'vacant']

vac_boc_j = boc_j(p_vac)
vac_boc_j.to_csv('vac_boc_jurisdiction.csv')

vac_boc_c = boc_c(p_vac)
vac_boc_c.to_csv('vac_boc_county.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct
juris_id_y,,,,,,,,,
alam,1248.085181,"1,700",520,"10,030,271","4,768,095","-1,180","-5,262,176",-0.694126,-0.524629
alba,432.557933,80,0,"260,259",0,-80,"-260,259",-1.000000,-1.000000
amer,532.289077,446,113,"5,435,865","3,851,284",-332,"-1,584,581",-0.745392,-0.291505
anti,7229.729103,"31,192","1,098","88,136,924","13,339,079","-30,093","-74,797,845",-0.964774,-0.848655
athe,83.556223,48,0,"424,797",0,-48,"-424,797",-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...
vall,9042.701236,"6,149",0,"14,978,338",0,"-6,149","-14,978,338",-1.000000,-1.000000
walc,3898.844618,"10,062","1,469","2,655,495","7,031,783","-8,592","4,376,287",-0.853928,1.648011
wind,1296.603117,"2,226","2,355","2,916,536","5,456,579",129,"2,540,043",0.058094,0.870911


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,COUNTY_ID,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct,ctyName
0,1.0,230468.562801,"73,908","184,765","291,007,238","942,379,164","110,856","651,371,926",1.499925,2.238336,Alameda
1,13.0,239812.143402,"159,961","48,887","345,373,168","203,381,799","-111,073","-141,991,368",-0.694379,-0.411124,Contra Costa
2,41.0,221327.899567,"22,343","109,975","16,814,875","276,089,590","87,631","259,274,714",3.921926,15.419365,Marin
3,55.0,95917.110035,"22,743","13,147","76,200,883","544,519,638","-9,596","468,318,755",-0.421932,6.145844,Napa
4,75.0,2012.375580,"5,826","4,000","40,126,515","38,441,118","-1,825","-1,685,396",-0.313364,-0.042002,San Francisco
5,81.0,200711.272259,"33,430","35,238","138,179,993","130,347,805","1,807","-7,832,188",0.054065,-0.056681,San Mateo
6,85.0,489742.499923,"219,027","125,295","1,785,023,932","221,510,032,626","-93,731","219,725,008,693",-0.427944,123.093593,Santa Clara
7,95.0,288686.289781,"43,417","29,274","243,969,664","99,031,488","-14,142","-144,938,176",-0.325743,-0.594083,Solano
8,97.0,481874.669405,"44,217","25,111","317,030,682","67,921,770","-19,105","-249,108,912",-0.432086,-0.785756,Sonoma


In [25]:
# low ILR parcel statistics (threadhold 0.2)
plu_main.ILR = pd.to_numeric(plu_main.ILR, errors='coerce')
p_low_ILR = plu_main.loc[plu_main.ILR < 0.2]

low_ILR_boc_j = boc_j(p_low_ILR)
low_ILR_boc_j.to_csv('low_ILR_boc_jurisdiction.csv')

low_ILR_boc_c = boc_c(p_low_ILR)
low_ILR_boc_c.to_csv('low_ILR_boc_county.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct
juris_id_y,,,,,,,,,
alam,229.266511,"1,019",194,"9,446,845","1,774,523",-824,"-7,672,321",-0.808908,-0.812157
alba,28.675618,529,0,"1,010,220",0,-529,"-1,010,220",-1.000000,-1.000000
anti,7610.578627,"28,829","1,074","95,461,433","12,604,231","-27,754","-82,857,202",-0.962726,-0.867965
athe,436.792943,411,0,"3,280,344",0,-411,"-3,280,344",-1.000000,-1.000000
belm,358.951603,825,638,"6,959,931","1,506,316",-187,"-5,453,615",-0.226707,-0.783573
...,...,...,...,...,...,...,...,...,...
vaca,4759.994793,"6,547","1,137","23,774,539","15,014,456","-5,410","-8,760,083",-0.826317,-0.368465
vall,2188.668326,"3,364",0,"17,946,778",0,"-3,364","-17,946,778",-1.000000,-1.000000
walc,4058.807474,"6,300","1,335","5,077,766","2,413,622","-4,965","-2,664,144",-0.788053,-0.524668


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,COUNTY_ID,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct,ctyName
0,1.0,175765.465815,"116,447","108,970","449,487,414","778,916,920","-7,477","329,429,506",-0.064211,0.732900,Alameda
1,13.0,280997.464041,"128,363","51,067","330,026,568","208,520,252","-77,296","-121,506,315",-0.602169,-0.368171,Contra Costa
2,41.0,74589.311165,"36,409","112,770","23,111,730","197,644,080","76,361","174,532,349",2.097315,7.551678,Marin
3,75.0,878.698191,"15,041","5,100","74,142,054","46,631,512","-9,941","-27,510,541",-0.660942,-0.371052,San Francisco
4,81.0,118639.711196,"31,575","22,712","195,425,935","77,676,119","-8,862","-117,749,815",-0.280687,-0.602529,San Mateo
5,85.0,371116.819941,"139,924","84,825","1,726,669,972","228,314,752,114","-55,098","226,588,082,142",-0.393778,131.228368,Santa Clara
6,95.0,304074.167364,"55,474","22,812","328,366,377","81,095,798","-32,662","-247,270,578",-0.588778,-0.753033,Solano
7,97.0,498976.883230,"46,085","50,423","286,534,595","195,325,867","4,337","-91,208,728",0.094123,-0.318317,Sonoma


In [26]:
# Old building parcel statistics (1930-1980)
plu_main.year_built = pd.to_numeric(plu_main.year_built, errors='coerce')
p_old = plu_main.loc[(plu_main.year_built < 1980) & (plu_main.year_built >= 1930)]

old_boc_j = boc_j(p_old)
old_boc_j.to_csv('old_boc_jurisdiction.csv')

old_boc_c = boc_c(p_old)
old_boc_c.to_csv('old_boc_county.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct
juris_id_y,,,,,,,,,
alam,3464.036126,"13,079","1,993","10,900,471","18,319,449","-11,086","7,418,978",-0.847599,0.680611
alba,2009.003610,"6,266",0,"11,222,450","14,707","-6,266","-11,207,742",-1.000000,-0.998689
amer,363.616684,"1,125",8,"943,056","811,824","-1,117","-131,232",-0.992510,-0.139156
anti,2692.749556,"20,044",540,"24,789,008","23,195,888","-19,504","-1,593,120",-0.973045,-0.064267
athe,2022.486361,"1,961",0,"15,300,412",0,"-1,961","-15,300,412",-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...
vall,5043.332009,"27,504",0,"38,460,202",0,"-27,504","-38,460,202",-1.000000,-1.000000
walc,5845.449394,"29,404","6,203","34,075,507","22,567,148","-23,200","-11,508,359",-0.789023,-0.337731
wind,954.799128,"3,377","3,319","2,322,464","6,424,861",-57,"4,102,397",-0.016993,1.766399


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,COUNTY_ID,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct,ctyName
0,1.0,114291.727364,"567,191","299,775","953,968,973","3,820,325,036","-267,416","2,866,356,063",-0.471474,3.004664,Alameda
1,13.0,111017.554884,"346,267","104,222","354,396,620","315,430,225","-242,044","-38,966,395",-0.699010,-0.109951,Contra Costa
2,41.0,127469.517135,"140,282","252,910","106,773,385","207,760,356","112,628","100,986,971",0.802871,0.945807,Marin
3,55.0,311599.693557,"317,660","41,641","228,165,223","1,193,422,289","-276,018","965,257,066",-0.868912,4.230518,Napa
4,75.0,9176.979954,"185,929","33,973","678,520,947","173,982,058","-151,956","-504,538,888",-0.817279,-0.743586,San Francisco
5,81.0,112244.589138,"281,996","90,069","524,681,447","227,115,659","-191,926","-297,565,787",-0.680600,-0.567136,San Mateo
6,85.0,218281.371334,"506,501","278,386","3,150,259,164","169,996,642,838","-228,115","166,846,383,673",-0.450375,52.962748,Santa Clara
7,95.0,167993.985438,"87,001","26,022","202,567,680","64,599,390","-60,978","-137,968,290",-0.700891,-0.681097,Solano
8,97.0,312131.304041,"140,886","103,869","714,834,607","225,933,107","-37,016","-488,901,500",-0.262743,-0.683937,Sonoma
